#Paquetes necesarios

In [1]:
import cv2  
import math 
from ultralytics import YOLO



AttributeError: 'OutStream' object has no attribute 'reconfigure'

Desde cámara, detección con yolov8 y modelo nano. Visualización propia

In [6]:
# Carga del modelo
model = YOLO('yolov8cars.pt') #Contenedores

# Nombre de las distintas clases
classNames = ['Ambulance', 'Bus', 'Car', 'Motorcycle', 'Truck']


# Captura desde lawebcam
vid = cv2.VideoCapture(0)
  
while(True):      
    # fotograma a fotograma
    ret, img = vid.read()
  
    # si hay imagen válida
    if ret:  
        # Perform inference on an image
        results = model(img, stream=True)
        
        # Para cada detección
        for r in results:
            boxes = r.boxes

            for box in boxes:
                # Contenedor
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values
                
                # Confianza
                confidence = math.ceil((box.conf[0]*100))/100
                print("Confidence --->",confidence)

                # Clase
                cls = int(box.cls[0])
                print("Class name -->", classNames[cls])

                # Convierte identificador numérico de clase a un color RGB
                escala = int((cls / len(classNames)) * 255 * 3)
                if escala >= 255*2:
                    R = 255
                    G = 255
                    B = escala - 255*2
                else:
                    if escala >= 255:
                        R = 255
                        G = escala - 255
                        B = 0
                    else:
                        R = escala
                        G = 0
                        B = 0

                # Dibuja el contenedor y clase
                cv2.rectangle(img, (x1, y1), (x2, y2), (R, G, B), 3)
                cv2.putText(img, classNames[cls] , [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)

        # Muestra fotograma
        cv2.imshow('Vid', img)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()

NameError: name 'YOLO' is not defined

Diversos modelos preentrenados, visualizando

In [7]:
# Carga del modelo
#model = YOLO('yolov8n.pt') #Contenedores
#model = YOLO('yolov8n-seg.pt') #Máscaras
model = YOLO('yolov8n-pose.pt')  #Pose

#Para un vídeo 
filename = "C:/Users/otsed/Desktop/RUNNERS_ILUSOS/Multimedia/Bibs/TGC23_PdH_C0056_resultado.mp4"
results = model(filename, show=True)

cv2.destroyAllWindows()



NameError: name 'YOLO' is not defined

Intregración con seguimiento (tracking)
Nota: he tenido que bajar a la versión de python 3.9.5 e instalar lap con pip install lap

In [ ]:
# Carga del modelo
model = YOLO('yolov8n.pt') #Contenedores
#model = YOLO('yolov8n-seg.pt') #Máscaras
#model = YOLO('yolov8n-pose.pt')  #Pose

#Para un vídeo 
#filename = "C:/Users/otsed/Desktop/RUNNERS_ILUSOS/Multimedia/Bibs/TGC23_PdH_C0056_resultado.mp4"
filename = "D:/GH010196s.mp4"
results = model.track(source=filename, show=True)  # BoT-SORT tracker (por defecto)
#results = model.track(source=filename, show=True, tracker="bytetrack.yaml")  # ByteTrack tracker

cv2.destroyAllWindows()

Reconocimiento de caracteres tras instalar pytesseract y tesseract

In [2]:
# Tesseract
import cv2
import pytesseract

# Previamente debes descargar los ejecutables
# Si la ruta de Tesseract no está en el PATH, ruta al ejecutable
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'

# Lenguajes disponibles
print(pytesseract.get_languages(config=''))

#Cargo imagen y ocnvierto a RGB
img = cv2.imread('toy.tif') 
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#Aplica reconocedor a imagen cargada
print(pytesseract.image_to_string(img_rgb))

['eng', 'osd']
Hasta el infinito y mas alla



Reconocimiento decaracteres tras instalar easyocr

In [3]:

# Carga de modelos YOLOv8
model_car = YOLO('yolov8cars.pt')
model_plate = YOLO('yolov8plates.pt')

# Configuración de Tesseract OCR
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'

# Clases de objetos para coches
class_names_car = ["car", "motorbike", "ambulance", "truck", "background"]

# Clases de objetos para matrículas
class_names_plate = ["license_plate", "background"]

# Captura desde un archivo de video en lugar de la webcam
video_path = 'cars1.mp4'
vid = cv2.VideoCapture(video_path)

while True:
    # Fotograma a fotograma
    ret, img = vid.read()

    # Si hay imagen válida
    if ret:
        # Perform inference on an image for coches
        results_car = model_car(img, stream=True)

        # Para cada detección de coche
        for r_car in results_car:
            boxes_car = r_car.boxes

            for box_car in boxes_car:
                # Contenedor de coche
                x1_car, y1_car, x2_car, y2_car = box_car.xyxy[0]
                x1_car, y1_car, x2_car, y2_car = int(x1_car), int(y1_car), int(x2_car), int(y2_car)

                # Clase de coche
                cls_car = int(box_car.cls[0])

                # Si la clase es un coche
                if class_names_car[cls_car] == "car":
                    # Perform inference on an image for matrículas
                    plate_region = img[y1_car:y2_car, x1_car:x2_car]
                    results_plate = model_plate(plate_region, stream=True)

                    # Para cada detección de matrícula
                    for r_plate in results_plate:
                        boxes_plate = r_plate.boxes

                        for box_plate in boxes_plate:
                            # Contenedor de matrícula
                            x1_plate, y1_plate, x2_plate, y2_plate = box_plate.xyxy[0]
                            x1_plate, y1_plate, x2_plate, y2_plate = int(x1_plate), int(y1_plate), int(x2_plate), int(y2_plate)

                            # Clase de matrícula
                            cls_plate = int(box_plate.cls[0])

                            # Si la clase es una matrícula
                            if class_names_plate[cls_plate] == "license_plate":
                                # Recorta la región de la matrícula (ajusta según tu necesidad)
                                gray_plate = cv2.cvtColor(plate_region[y1_plate:y2_plate, x1_plate:x2_plate], cv2.COLOR_BGR2GRAY)

                                # Aplica Tesseract OCR
                                plate_text = pytesseract.image_to_string(gray_plate)

                                # Imprime el resultado en consola
                                print("Matrícula:", plate_text)

                    # Dibuja el contenedor y clase de coche
                    cv2.rectangle(img, (x1_car, y1_car), (x2_car, y2_car), (0, 255, 0), 3)
                    cv2.putText(img, class_names_car[cls_car], [x1_car, y1_car], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        # Muestra fotograma
        cv2.imshow('Vid', img)

    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break

# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()




0: 256x416 (no detections), 615.2ms
Speed: 133.8ms preprocess, 615.2ms inference, 62.5ms postprocess per image at shape (1, 3, 256, 416)

0: 256x416 (no detections), 34.1ms
Speed: 2.0ms preprocess, 34.1ms inference, 8.1ms postprocess per image at shape (1, 3, 256, 416)

0: 256x416 (no detections), 35.2ms
Speed: 2.0ms preprocess, 35.2ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 416)

0: 256x416 (no detections), 47.8ms
Speed: 1.0ms preprocess, 47.8ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 416)

0: 256x416 (no detections), 36.3ms
Speed: 1.0ms preprocess, 36.3ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 416)

0: 256x416 (no detections), 50.7ms
Speed: 0.0ms preprocess, 50.7ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 416)

0: 256x416 1 Car, 51.4ms
Speed: 0.0ms preprocess, 51.4ms inference, 290.7ms postprocess per image at shape (1, 3, 256, 416)

0: 256x416 3 Cars, 54.5ms
Speed: 0.0ms preprocess, 54.5ms inference, 0.

In [ ]:
import easyocr

#Carga del modelo de lengua
reader = easyocr.Reader(['es']) 

#Reconocimiento de una imagen
result = reader.readtext('toy.tif')
print(result)

#Con restricción de caracteres reconocibles
#result = reader.readtext('toy.tif', allowlist ='0123456789')